In [1]:
import pandas as pd
import json

## Constants

In [2]:
# CONSTANTS

# Object template to communicate data to the CM

TEMPLATE = {
    "id": None,
    "userid": None,
    "origin": None,
    "source_id": "fake90e6d701748f08514b01",
    "source": "fake",
    "pname": None,
    "pvalue": None,
    "context": "application",
    "datapoints": 0,
    "category": None
}

FOLDER = "./data/"


C_CONTRIB = dict(
    datafile = 'DMH_127-AudioRecorded-DEGARI-Values.json',
    userIdAtt = 'User ID',
    artifactIdAtt = 'Object number',
    contribIdAtt = '_id',
    contribAtts = ['English translation','degari_extracted_emotions','plutchik_emotions','emotion_recognition','subjectivity_detection', 'polarity_classification','intensity_ranking','MFT_Values','EkmanEmotions'],
    ugcfile = "ugcContributions.json"
)

_ID = 0

## Users

It transforms demographic data from a LDH dataset into the ugc data sent to the CM by the UM

In [3]:
# User constants

C_USER =  dict(
    datafile ='DMH_127-AudioRecorded-DEGARI.json',
    demographicAtts = ['End-user community'],
    explicitCommunityAtt = 'End-user community',
    userIdAtt = 'User ID',
    ugcfile = "ugcUsers.json"
)

In [4]:
with open(FOLDER+C_USER["datafile"],encoding='utf-8') as f:
    userData = json.load(f)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 24311: character maps to <undefined>

In [ ]:
userData[0]

In [ ]:
postDict= {}
for user in userData:  
    user = user["0"]
    result = []
    currentData = TEMPLATE.copy()
    currentData["category"] = "demographics"
    currentData["userid"] = user[C_USER["userIdAtt"]]
    for item in C_USER["demographicAtts"]:
        if item in user:
            attData = currentData.copy()
            attData["id"] = _ID
            _ID+=1
            attData["origin"] = attData["userid"]
            attData["pvalue"] = user[item]
            item = "explicit-community" if item == C_USER["explicitCommunityAtt"] else item
            attData["pname"]  = item 
            result.append(attData)
    postDict[user[C_USER["userIdAtt"]]] = result

In [ ]:
with open(FOLDER+C_USER["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)

In [ ]:
# postDict is a dictionary with userids as fields and arrays of demographic data for the CM as values
# To POST users to a CM server in localhost:8080 uncomment next lines
# server = "http://localhost:8080"
# for key, value in postDict.items():
#    response=requests.post(f'{server}/v1.1/users/{key}/update-generated-content', json = value)
#    procesar el resultado de response

## Contributions


In [ ]:
with open(FOLDER+C_CONTRIB["datafile"]) as f:
    contribData = json.load(f)

In [ ]:
postDict= { att: {} for att in C_CONTRIB["contribAtts"]}

In [ ]:
for contrib in contribData:  
    contrib = contrib["0"]
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    currentData["origin"] = contrib[C_CONTRIB["artifactIdAtt"]]
    for interactionAtt in C_CONTRIB["contribAtts"]:
        if interactionAtt in contrib:
            interactionData = currentData.copy()
            interactionData['pname'] = interactionAtt
            interactionData['pvalue'] = contrib[interactionAtt]
            if contrib[C_CONTRIB["userIdAtt"]] in postDict[interactionAtt]:
                postDict[interactionAtt][contrib[C_CONTRIB["userIdAtt"]]].append(interactionData)
            else:
                postDict[interactionAtt][contrib[C_CONTRIB["userIdAtt"]]] = [interactionData]

In [ ]:
postDict

# Change pname to allow the CM to know the text associated to the interaction

In [ ]:
changeNameDict = {}
changeNameDict['English translation'] = 'text'

In [ ]:
postDict2 = {}
for key in postDict:
    if (key in changeNameDict):
        for userid in postDict[key]:
            for interaction in postDict[key][userid]:
                interaction['pname'] = changeNameDict[key]
        postDict2[changeNameDict[key]] = postDict[key]
    else:
        postDict2[key] = postDict[key]
postDict = postDict2 

In [ ]:
# Save the file

In [ ]:
with open(FOLDER+C_CONTRIB["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)